In [ ]:
'''
schematic overview of inference (regression)
'''

means_list, logvar_list =[], []
for batch in range(int(np.ceil(X_range.shape[0] / self.batch_size))):
    x_cat = X_cat[self.batch_size * batch: self.batch_size * (batch + 1)].cuda()
    x_range = X_range[self.batch_size * batch: self.batch_size * (batch + 1)].cuda()
    MC_samples = [self.predict_one(x_cat, x_range, stop_dropout=stop_dropout) for _ in range(K)]
    means_list.append(torch.stack([tup[0] for tup in MC_samples]).view(K, x_range.shape[0]))
    logvar_list.append(torch.stack([tup[1] for tup in MC_samples]).view(K, x_range.shape[0]))
means = torch.cat([el for el in means_list], dim=1).cpu().data.numpy()
logvar = torch.cat([el for el in logvar_list], dim=1).cpu().data.numpy()

epistemic_uncertainty = np.var(means, 0)
if heteroscedastic:
    aleatoric_uncertainty = np.exp(logvar)  
else:
    aleatoric_uncertainty = np.zeros(logvar.shape)

In [ ]:
'''
schematic overview of inference (classification)
'''

means_list, logvar_list =[], []
for batch in range(int(np.ceil(X_range.shape[0] / self.batch_size))):
    x_cat = X_cat[self.batch_size * batch: self.batch_size * (batch + 1)].cuda()
    x_range = X_range[self.batch_size * batch: self.batch_size * (batch + 1)].cuda()
    MC_samples = [self.predict_one(x_cat, x_range, stop_dropout) for _ in range(K)]
    means_list.append(torch.stack([tup[0] for tup in MC_samples]).view(K, x_range.shape[0], self.C))
    logvar_list.append(torch.stack([tup[1] for tup in MC_samples]).view(K, x_range.shape[0]))#.cpu().data.numpy())
means = torch.cat([el for el in means_list], dim=1)
logvar = torch.cat([el for el in logvar_list], dim=1).cpu().data.numpy()

means = F.softmax(means, dim=2).cpu().data.numpy()
predictive_entropy, mutual_information = calc_entropy(means)
aleatoric_uncertainty = (predictive_entropy - mutual_information).cpu().data.numpy()
epistemic_uncertainty = mutual_information.cpu().data.numpy()

In [ ]:
def calc_entropy(mc_outputs):
    '''
    computes predictive entropy and mutual information
    input 3-D Torch Tensor
    Dimension 1: K Monte Carlo samples per case in validation set
    Dimension 2: cases in validation set
    Dimension3: C classes
    Values should be in softmax along D3
    '''
    mc_outputs = torch.Tensor(mc_outputs)

    mc_outputs_avg = torch.mean(mc_outputs, dim=0)
    predictive_entropy = Categorical(probs=mc_outputs_avg).entropy()

    entropy = Categorical(probs=mc_outputs).entropy()
    expected_entropy = torch.mean(entropy, dim=0)

    mutual_information = predictive_entropy-expected_entropy
    return predictive_entropy, mutual_information